In [3]:

from art.attacks.evasion import CarliniL2Method, HopSkipJump
from art.estimators.classification import PyTorchClassifier, BlackBoxClassifier
import torch
import timm
from torchvision import transforms
from PIL import Image

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
target_model = timm.create_model('vit_base_patch16_224.augreg_in21k_ft_in1k', pretrained=True)
target_model.head = torch.nn.Linear(target_model.head.in_features, 2)
target_model = target_model.to(device)
target_model.load_state_dict(
    torch.load(
        "../weights/vit_teacher.pth"
    )
)
target_model.eval()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_12680\3032653916.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [6]:
# DO NOT CHANGE
labels = {}
labels[0] = "fake"
labels[1] = "real"
classifier = PyTorchClassifier(
    model = target_model,
    loss = torch.nn.CrossEntropyLoss(),
    nb_classes = len(labels), # 1000
    input_shape = (3, 224, 224)
)

In [7]:
attack = CarliniL2Method(classifier)

In [9]:
img = Image.open("../static/fake/0001_00_00_01_0.jpg")
preprocess = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
]);

unnormalize = transforms.Normalize(
   mean= [-m/s for m, s in zip([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])],
   std= [1/s for s in [0.229, 0.224, 0.225]]
)

img_tensor = preprocess(img).unsqueeze(0)
# run the attacks
adversarial_example = attack.generate(img_tensor.numpy())

C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
# DO NOT CHANGE

adv_tensor = torch.from_numpy(adversarial_example).to(device)
output = target_model(adv_tensor)

print(f"Output index:\n---------------\n{output[0].argmax()}\n")
print(f"Output label:\n---------------\n{labels[output[0].argmax().item()]}\n")

Output index:
---------------
1

Output label:
---------------
real



In [16]:
def tensor_to_pil(img_tensor):    
    unnormed_tensor = unnormalize(img_tensor)
    return transforms.functional.to_pil_image(unnormed_tensor[0])
masked_pil= tensor_to_pil(adv_tensor)
masked_pil.save(fp="../static/adversarial/output_art.png")